In [1]:
import os
import base64
import sys
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.fernet import Fernet
from base64 import b64encode, b64decode

#CODE REFERENCE: https://nitratine.net/blog/post/encryption-and-decryption-in-python/#generating-a-key-from-a-password
'''
UNIVERSIDAD DEL VALLE DE GUATEMALA
CIFRADO DE INFORMACION
GRUPO: 6 ->
AMADO GARCIA 
SARA ZAVALA
RICARDO VALENZUELA
ANDREE TOLEDO
JUAN FERNANDO DE LEON
'''

#Part 2, exercise 1
class TextCipher(object):
    
    #This method returns a tuple, which are the encryption key and the salt value.
    #The function takes 1 argument, the uncoded password written by the user
    def encryption_key(self, password):
        encoded_password = password.encode() #This encodes, to bytes, the password
        salt = os.urandom(16) #Returns a random value of 16 bytes
        kdf = PBKDF2HMAC( #Key derivation Function(algorithm, lenght, salt, iterations, backend)
            algorithm=hashes.SHA256(), 
            length=32,
            salt=salt, #This is a random data that hashes the provided password
            iterations=100000,
            backend=default_backend()
        )
        #This is the resulting key after being proccessed by the kdf and encoded to bytes
        key = base64.urlsafe_b64encode(kdf.derive(encoded_password)) 

        return key, salt
    
     #This method returns the key used to encrypt a text
     #The function takes 2 arguments, the password and the salt.
     #The password must be the same so the key is the same as well
    def decryption_key(self, password, salt):
        encoded_password = password.encode()  #This encodes, to bytes, the password
        salt = b64decode(salt) #Decodes the b64 value of salt to bytes
        kdf = PBKDF2HMAC(       #Key derivation Function(algorithm, lenght, salt, iterations, backend)
            algorithm=hashes.SHA256(),
            length=32,
            salt=salt, #This is a random data that hashes the provided password
            iterations=100000,
            backend=default_backend()
        )
        #This is escencially the same key as the encryption one, if the user provided the same password
        key = base64.urlsafe_b64encode(kdf.derive(encoded_password)) 
        return key
    
    #Encryption function, takes 2 arguments: password and a string
    #Returns a dictionary with a ciphered text and the salt
    def encrypt(self, password, plaintext):
        key, salt = self.encryption_key(password) #calls the encryption_key function to get the key and 
                                                  #salt values

        message = plaintext.encode() #Encodes to bytes the provided message string
        
        #Fernet guarantees that a message encrypted using it can't be manipulated or read without the key.
        f = Fernet(key) 
        
        #This is the encrypted message, using the encrypt function provided by Fernet library
        encrypted = f.encrypt(message)
        return {
        'cipher_text': encrypted.decode('utf-8'),
        'salt': b64encode(salt).decode('utf-8'),
        }
         
    #Decryption function, takes 2 arguments: password and a dictionary with keys -> cipher_text and salt
    #Returns a utf-8 type string with the decrypted value
    def decrypt(self, password, encrypted_dict):
        #calls the decryption_key function to get the key used to encrypt the text. Password must match
        key = self.decryption_key(password, encrypted_dict['salt'])
        f = Fernet(key) #Same as described above
        #Fernet provides a decrypt function for encoded text, with the correct key. We also decoded for
        #usability pourposes
        try:
            decrypted = f.decrypt(encrypted_dict['cipher_text'].encode()).decode('utf-8')
            return decrypted
        except:
            print('Wrong password provided')
          
            
        
    
'''
PREGUNTAS:
i. ¿Tuvo que usar “encode” de algo? ¿Sobre qué variables?
    Sí, usamos encode en varias variables: 
    1) password
    2) key 
    3) El mensaje en texto plano
    4) El mensaje encriptado
    5) La variable Salt (base64)
 
ii. ¿Qué modo de AES usó? ¿Por qué?
Referencias: https://stackoverflow.com/questions/1949640/does-iv-work-like-salt
https://pycryptodome.readthedocs.io/en/latest/src/cipher/classic.html#cbc-mode
    Se utilizó PBE (Password Base Encryption) con CBC (Ciphertext Block Chainin), ya que
    se devuelve un iv (salt) y el texto encriptado, cuyos valores se utilizan par desencriptar,
    tal y como se describe en el algoritmo CBC tradicional. Ya que se nos pedía un algoritmo que
    implementara el uso de una contraseña, esté fue el modo utilizado.

iii. ¿Qué parámetros tuvo que hacer llegar desde su función de Encrypt a la Decrypt? ¿Por qué?
      Devolví desde mi función Encrypt un diccionario con el texto cifrado y el "salt", ya 
      que que Salt es generado de forma aleatoria, era necesario mantener el mismo valor para que al hacer
      el hash con el password obtuvieramos la misma key. El texto cifrado lógicamente era necesario para
      poder desencriptarlo.
'''
    


'\nPREGUNTAS:\ni. ¿Tuvo que usar “encode” de algo? ¿Sobre qué variables?\n    Sí, usamos encode en varias variables: \n    1) password\n    2) key \n    3) El mensaje en texto plano\n    4) El mensaje encriptado\n    5) La variable Salt (base64)\n \nii. ¿Qué modo de AES usó? ¿Por qué?\nReferencias: https://stackoverflow.com/questions/1949640/does-iv-work-like-salt\nhttps://pycryptodome.readthedocs.io/en/latest/src/cipher/classic.html#cbc-mode\n    Se utilizó PBE (Password Base Encryption) con CBC (Ciphertext Block Chainin), ya que\n    se devuelve un iv (salt) y el texto encriptado, cuyos valores se utilizan par desencriptar,\n    tal y como se describe en el algoritmo CBC tradicional. Ya que se nos pedía un algoritmo que\n    implementara el uso de una contraseña, esté fue el modo utilizado.\n\niii. ¿Qué parámetros tuvo que hacer llegar desde su función de Encrypt a la Decrypt? ¿Por qué?\n      Devolví desde mi función Encrypt un diccionario con el texto cifrado y el "salt", ya \n    

In [2]:
import os
  
#THIS FILE CIPHER ONLY WORKS WITH .TXT FILES
#https://nitratine.net/blog/post/encryption-and-decryption-in-python/#generating-a-key-from-a-password
'''
UNIVERSIDAD DEL VALLE DE GUATEMALA
CIFRADO DE INFORMACION
GRUPO: 6 ->
AMADO GARCIA 
SARA ZAVALA
RICARDO VALENZUELA
ANDREE TOLEDO
JUAN FERNANDO DE LEON
'''

#I'm using the encryption and decryption methods from my TextCipher class in the texcipher.py file
#The main difference is that I write out the results in a txt file
class FileCipher(object):

    def __init__(self):
        self.text_cipher = TextCipher()
        self.text = None
    
    #This Method is in charge of encrypting the data that is been read from the file
    #The function takes 3 arguments: The password, the input file name and the output file name (and extensions)
    def encrypt(self, password, input_file, output_file):
        self.text = self.__read(input_file, True)
        if self.text is not None:
            encrypted = self.text_cipher.encrypt(password, self.text.decode())
            self.__write(encrypted, True, output_file)
            return True
        return False
    
    #This Method is in charge of decrypting the data that is been read from the file
    #The function takes 3 arguments: The password, the input file name and the output file name (and extensions)
    def decrypt(self, password, input_file, output_file):
        self.text = self.__read(input_file, False)
        text_dict = {'cipher_text': self.text[0].decode(), 'salt': self.text[1].decode()}
        decrypted = self.text_cipher.decrypt(password, text_dict)
        if decrypted is not None:
            self.__write(decrypted, False, output_file)
            return True
        return False
    
    #Private Method used to write the output (encrypted or decrypted) file. 
    #It takes 3 arguments, the text (encrypted or plain), a boolean indicating wheter it is encrypting or not
    #and the output file name and extension.
    def __write(self, text, isEncrypting, output_file):
        if isEncrypting:
            with open(output_file, 'wb') as f:
                f.write((text['cipher_text'] + "\n").encode())
                f.write(text['salt'].encode())
        else:
             with open(output_file, 'w') as f:
                f.writelines(text)
       
    #Private Method used to read the input (encrypted or decrypted) file. 
    #It takes 2 arguments, a boolean indicating wheter it is encrypting or not
    #and the input file name and extension. It returns the data in the file.
    def __read(self,  input_file, isEncrypting,):
        try:
            if isEncrypting:
                with open(input_file, 'rb') as f:
                    data = f.read()
            else:
                with open(input_file, 'rb') as f:
                    data = f.readlines()
            return data
        except:
            print('This file does not exist')
            
        
'''
i. ¿Qué modo de AES usó? ¿Por qué?
    Referencias: https://stackoverflow.com/questions/1949640/does-iv-work-like-salt
    https://pycryptodome.readthedocs.io/en/latest/src/cipher/classic.html#cbc-mode
    Se utilizó PBE (Password Base Encryption) con CBC (Ciphertext Block Chainin), ya que
    se devuelve un iv (salt) y el texto encriptado, cuyos valores se utilizan par desencriptar,
    tal y como se describe en el algoritmo CBC tradicional. Ya que se nos pedía un algoritmo que
    implementara el uso de una contraseña, esté fue el modo utilizado.
    
ii. ¿Qué parámetros tuvo que hacer llegar desde su función de Encrypt a la Decrypt? ¿Por
    1. Password
    2. Input file
    3. Ouput file
    Al igual que en la encriptación de texto, necesitaba saber el Salt y el text encriptado, ya que
    la lógica era la misma, con la diferencia de que el text obtenido se guarda en un archivo .txt.
    La lectura y escritura se raliza en binario.
    
qué?
iii. ¿Qué variables considera las más importantes dentro de su implementación? ¿Por qué?
     1. Password
     2. Input file 
     3. Output file
     4. Text
     Ya que estos son los parámetros pasados en las funciones y que el cipher de texto se encarga del texto,
     lo mas importante era establecer un password, conocer el nombre del archivo a leer, obtener el texto (plano),
     luego el nombre del archivo a escribir y nueva mente el texto (Cifrado) y así el proceso inverso al momento
     de desencriptar.
'''
        

'\ni. ¿Qué modo de AES usó? ¿Por qué?\n    Referencias: https://stackoverflow.com/questions/1949640/does-iv-work-like-salt\n    https://pycryptodome.readthedocs.io/en/latest/src/cipher/classic.html#cbc-mode\n    Se utilizó PBE (Password Base Encryption) con CBC (Ciphertext Block Chainin), ya que\n    se devuelve un iv (salt) y el texto encriptado, cuyos valores se utilizan par desencriptar,\n    tal y como se describe en el algoritmo CBC tradicional. Ya que se nos pedía un algoritmo que\n    implementara el uso de una contraseña, esté fue el modo utilizado.\n    \nii. ¿Qué parámetros tuvo que hacer llegar desde su función de Encrypt a la Decrypt? ¿Por\n    1. Password\n    2. Input file\n    3. Ouput file\n    Al igual que en la encriptación de texto, necesitaba saber el Salt y el text encriptado, ya que\n    la lógica era la misma, con la diferencia de que el text obtenido se guarda en un archivo .txt.\n    La lectura y escritura se raliza en binario.\n    \nqué?\niii. ¿Qué variables 

In [ ]:
'''
Main Class
UNIVERSIDAD DEL VALLE DE GUATEMALA
CIFRADO DE INFORMACION
GRUPO: 6 ->
AMADO GARCIA 
SARA ZAVALA
RICARDO VALENZUELA
ANDREE TOLEDO
JUAN FERNANDO DE LEON
'''

while(True):
    menu = 'Press 1 To Encrypt and Decrypt a Text\n' + 'Press 2 To Encrypt and Decrypt a .txt File\n' + 'Press anything else to exit\n'
 
    option = input(menu)
    
    if option == '1':
        t_c = TextCipher()

        plain_text = input('Enter the text you want to cypher: ')
        password = input('Enter your password: ')

        encrypted_message = t_c.encrypt(password, plain_text)
        print('Your encrypted messages is: ' + str(encrypted_message) + '\n')

        password = input('Enter your password to decrypt: ')
        decrypted_message = t_c.decrypt(password, encrypted_message)
        print('Your decrypted message is: ' + decrypted_message)

    elif option == '2':
        f_c = FileCipher()  

        pwd = input('Insert your password: ')
        input_file = input('Insert your input filename without the .txt extension')
        output_file = input('Insert your output filename without the .txt extension')
        isValid = f_c.encrypt(pwd, input_file + '.txt', output_file + '.txt')
        if isValid:
            print('Encryption succesful!')
            pwd = input('Insert again your password to decrypt: ')
            isValid = f_c.decrypt(pwd, output_file + '.txt', 'decrypted.txt')
            if isValid:
                print('Decryption succesful, saved on decrypted.txt')
    else:
        print('Good bye')
        break

Press 1 To Encrypt and Decrypt a Text
Press 2 To Encrypt and Decrypt a .txt File
Press anything else to exit
1
Enter the text you want to cypher: El Curso de cifrado es lo máximo 
Enter your password: 54321
Your encrypted messages is: {'cipher_text': 'gAAAAABfJ5bkC49wAlUquZWM9ZW2RT-sa9gXtaTkqC8DXa-fReXfUhO9ufvJQeHDE04ZAChZ4b55PLmCPs84wKy3e9sSk5P9kj8zn3zsezQVXiIGDs_q0zgmb6M1nfHvJUbTaLb1nPrZ', 'salt': 'tPMezO36yk7ol9pTnXSz3Q=='}

Enter your password to decrypt: 54321
Your decrypted message is: El Curso de cifrado es lo máximo 
Press 1 To Encrypt and Decrypt a Text
Press 2 To Encrypt and Decrypt a .txt File
Press anything else to exit
2
Insert your password: 9985
Insert your input filename without the .txt extensionplain_text
Insert your output filename without the .txt extensionstory
Encryption succesful!
Insert again your password to decrypt: 9985
Decryption succesful, saved on decrypted.txt
